<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/llibre-estil/logo-UOC-2linies.png", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.859 · Visualización de datos</p>
<p style="margin: 0; text-align:right;">2023-1 · Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

<br><p style='font-size: 50px;'>Visualización de datos - PEC3</p><br><br>

<div class="alert alert-block alert-info">
    <strong>Nombre y apellidos:</strong> Juan Gabriel Marinero Tarazona
</div>
<strong></strong>

<div id="table-of-content"></div>

##### imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import copy

%matplotlib inline

In [ ]:
from os.path import dirname, basename, isfile, join
from importlib import import_module
import glob
import os

current_dir = os.getcwd()
modules = glob.glob(join(current_dir+ '/ML_fcns', "*.py"))
module_names = [basename(f)[:-3] for f in modules if isfile(f) and not f.endswith('__init__.py')]
for module_name in module_names:
    module = import_module(f"ML_fcns.{module_name}")

    # Add the imported module to __all__
    globals()[module_name] = getattr(module, module_name)
    __all__ = module_names

In [ ]:
display_htmlFcn2 = lambda *args, **kwargs: display_htmlFcn(*args, _hide_index=True,  
                                                           _hide_columns=True, _space=5, **kwargs)

In [ ]:
from ML_fcns.display_htmlFcn_applyMap_HideVals import display_htmlFcn_applyMap_HideVals as hideVals
hideZeros = lambda val: hideVals(val,0)

In [ ]:
def softZeros(val):
    return 'color:#D3D3D3;background-color:white' if val==0 else ''

In [ ]:
def highlight_maxFcn(s, props='color:black;background-color:gold'):
    return np.where(s == np.nanmax(s.values), props, '')

In [ ]:
def plotNN02(weights_history, biases_history, figsize=(12,5), radialBool=False, kwargs=dict()):
    
    if isinstance(weights_history, dict):
        i = list(weights_history.keys())[-1] # key_last_iter
        w_i, w_j = [weights_history.get(i).get(k) for k in ['w_1','w_2']]
        b_i, b_j = [biases_history.get(i).get(k)  for k in ['b_1','b_2']]
    else:
        w_i, w_j = [weights_history[-1].get(k) for k in ['w_1','w_2']]
        b_i, b_j = [biases_history[-1].get(k)  for k in ['b_1','b_2']]

    # making of:
    # plotNN01([2,4], weights=np.array(w_i)[np.newaxis,:], biases=b_i, figsize=(10,3), radialBool=False);
    # plotNN01([4,1], weights=np.array(w_j)[np.newaxis,:], biases=b_j, figsize=(5,3), radialBool=False);
    
    w_i = np.array(w_i)[np.newaxis,:][0]
    b_i = np.array(b_i)[0]
    w = [w_i]
    b = [b_i]
    if w_j is not None:
        w_j = np.array(w_j)[np.newaxis,:][0]
        b_j = np.array(b_j)[0]
        w.append(w_j)
        b.append(b_j)

    cells = [len(w[0])]
    cells.extend([len(k) for k in b])


    return plotNN01(cells, weights=w, biases=b, figsize=figsize, radialBool=radialBool, 
                    arrow_opacity_fcn=lambda x: 0.3, **kwargs)

# A Gentle Introduction to Artificial Neural Networks

Main <a href="https://dustinstansbury.github.io/theclevermachine/a-gentle-introduction-to-neural-networks">reference</a>.

## Single-layer Neural Networks

<img align="center" style="width:300px;padding-left:45px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/perceptron2.png">

### activation function

$$g_{linear}(z)=z$$

which outputs continuous values $a_{linear} \in (-\infty,\infty)$
, then the network implements a linear model akin to used in standard linear regression.



$$g_{logistic}(z)=\dfrac{1}{1+e^{-z}}$$

which outputs values $a_{logistic} \in (0,1)$

Binary classification can also be implemented using the **hyperbolic tangent** function

$$tanh(z)$$

which outputs values $a_{tanh} \in (−1,1)$.

In [ ]:
display_htmlFcn_plots_activation_fcn_statics()

In [ ]:
display_htmlFcn_plots_activation_fcn_interactive()

- The **logistic** and **tanh** activation functions (blue and green) have the quintessential sigmoidal “s” shape that **saturates for inputs of large magnitude**. This behavior makes them useful for **categorization**. 
- The **identity  linear** activation (red), however forms a **linear mapping** between the input to the activation function, which makes it useful for **predicting continuous** values. Example [here](#Neural-Networks-for-Regression).

A key property of these activation functions is that they are all smooth and differentiable. We’ll see later in this post why <u>differentiability is important for training neural networks</u>. The derivatives for each of these common activation functions are given by (for mathematical details on calculating these derivatives, see this [post](https://dustinstansbury.github.io/theclevermachine/derivation-common-neural-network-activation-functions)):

$$
\begin{array}{l}
g′_{linear}(z)&=1\\
g′_{logistic}(z)&=g_{logistic}(z)(1−g_{logistic}(z))\\
g′_{tanh}(z)&=1−g^2_{tanh}(z)
\end{array}
$$


These **derivatives** are either a constant (i.e. 1), or are can be **defined in terms of the original function**. This makes them extremely convenient for **efficiently** training neural networks, as we can implement the gradient using **simple manipulations of the feed-forward states of the network**.

## Multi-layer Neural Networks

<figure>
    <img align="center" style="width:500px;padding-left:45px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/multi-layer-perceptron.png">
</figure>

Multi-layer neural networks form compositional functions that map the inputs nonlinearly to outputs. If we associate:
- index $i$ with the input layer, 
- index $j$ with the hidden layer, and 
- index $k$ with the output layer, 

$$
a_{out}=a_k
=g_k (z_k)
=g_k \left(b_k+\sum_j a_j      \cdot w_{jk}\right)
=g_k \left(b_k+\sum_j g_j(z_j) \cdot w_{jk}\right)
=g_k \left(b_k+\sum_j g_j\left(b_j+\sum_i a_i w_{ij}\right)w_{jk}\right)
$$

where,

- $z_l$ is the pre-activation values for the units in layer $l$    
- $g_l()$ is the activation function for units in layer $l$ (assuming the same function for all units)
- $a_l=g_l(z_l)$ is the output activation for units in layer $l$
- $w_{l−1,l}$ are the parameters that weight the output messages of units feeding into layer $l$ to the activation function of units for that layer.
- $b_l$ term is the bias/DC offset for units in layer $l$

### Training neural networks & gradient descent

Training neural networks involves determining the model parameters $\theta=\{w,b\}$
that minimize the errors the network makes. This first requires that we have a way of quantifying error. A standard way of quantifying error is to take the squared difference between the network output and the target value:

$$
E=\dfrac{(output−target)^2}{2}
$$

With an **error function** in hand, we then aim to **find the setting of parameters that minimizes this error function**, when aggregated across all the training data. This concept can be interpreted spatially by imagining a “parameter space” whose dimensions are the values of each of the model parameters, and for which the error function will form a surface of varying height depending on its value for each parameter. Model <u>training is thus equivalent to finding point in parameter space (like a meshgrid) that makes the height of the error surface small</u>.

## Analysis of simple neural networks

### Single-layer neural network

<img align="right" style="width:600px;padding-left:45px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/single-layer-ann-error-surface.png">

To get a better intuition behind the concept of minimizing the error surface, let’s define a super-simple neural network, one that has a single input and a single output. For further simplicity, we’ll assume the network has no bias term and thus has a single parameter, $w_1$. We will also assume that the output layer uses the logistic sigmoid activation function. Accordingly, the network will map some input value $a_0$ onto a predicted output $a_{out}$ via the following function.

$$
a_{out}=g_{logistic}(a_0 w_1)
$$

Now let’s say we want this simple network to learn the identity function: given an input of 1 it should return a target value of 1. Given this target value we can now calculate the value of the error function for each setting of $w_1$. Varying the value of $w_1$ from -10 to 10 results in the error surface displayed in the right of Figure 4. We see that:
- the error is small for large positive values of $w_1$, 
- while the error is large for strongly negative values of $w_1$. 

This not surprising, given that the output activation function is the logistic sigmoid, which will map large values onto an output of 1.

In [ ]:
from collections import OrderedDict

# Define a few common activation functions
g_linear = lambda z: z
g_sigmoid = lambda z: 1./(1. + np.exp(-z))
g_tanh = lambda z: np.tanh(z)
   
# ...and their analytic derivatives    
g_prime_linear = lambda z: np.ones(len(z))
g_prime_sigmoid = lambda z: 1./(1 + np.exp(-z)) * (1 - 1./(1 + np.exp(-z)))
g_prime_tanh = lambda z: 1 - np.tanh(z) ** 2

activation_functions = OrderedDict(
    [
        ("linear", (g_linear, g_prime_linear, 'red')),
        ("sigmoid", (g_sigmoid, g_prime_sigmoid, 'blue')),
        ("tanh", (g_tanh, g_prime_tanh, 'green')),
    ]
)

In [ ]:
parameter_range_arr = [5,10,30]  
fig = plt.figure(figsize=(12, 5))
for i,k in enumerate(parameter_range_arr):
    ax = fig.add_subplot(1, len(parameter_range_arr), i+1, projection='3d')
    plotSurfaceError_01(ax, k)
plt.suptitle('Single-layer Network\nError Surface', fontsize=18);

In [ ]:
plotSurfaceError_02()

In [ ]:
plotSurfaceError_03()

### Multi-layer neural network

<img align="right" style="width:600px;padding-left:45px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/multi-layer-ann-error-surface.png">
Things become more interesting when we move from a single-layered network to a multi-layered network. Let’s repeat the above exercise, but include a single hidden node between the input and the output. Again, we will assume no biases, and logistic sigmoid activations for both the hidden and output nodes. Thus the network will have two parameters: $(w_1,w_2)$. Accordingly the 2-layered network will predict an output with the following function:

$$
a_{out}=g_{logistic}\left(g_{logistic}(a_0 w_1)w_2\right)
$$

Now, if we vary both $w_1$ and $w_2$, we obtain the error surface in right of Figure 5.

We see that the error function is minimized when both $w_1$ and $w_2$ are large and positive. We also see that the error surface is more complex than for the single-layered model, exhibiting a number of wide plateau regions. It turns out that the error surface gets more and more complicated as you increase the number of layers in the network and the number of units in each hidden layer. Thus, it is important to consider these phenomena when constructing neural network models.

The examples in Figures 4-5 gives us a qualitative idea of how to train the parameters of an NN, but we would like a more automatic way of doing so. Generally this problem is solved using **gradient descent**. The gradient descent algorithm (lee lo de Montecarlo en asig. eléctrica SEE):
- first calculates the derivative / gradient of the error function with respect to each of the model parameters. This gradient information will give us the direction in parameter space that decreases the height of the error surface.
- We then take a step in that direction 
- and repeat, iteratively calculating the gradient and taking steps in parameter space.

In [ ]:
def two_layer_network_predict(w1, w2, target_value=1):
    return g_sigmoid(w2 * g_sigmoid(w1 * target_value))

parameter_range_arr = [5,10,30]  
fig = plt.figure(figsize=(12, 5))
for i,k in enumerate(parameter_range_arr):
    ax = fig.add_subplot(1, len(parameter_range_arr), i+1, projection='3d')
    plotSurfaceError_01(ax, k, layer_network_predict=two_layer_network_predict, set_xlabel_bool=True,
                    view_angle=[25, 45])
plt.suptitle('Multi-layer Network\nError Surface', fontsize=18);

### The Backpropagation Algorithm

<img align="right" style="width:500px;padding-left:45px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/backpropagation-steps.png">
<figcaption>Figure 3: Diagram of a multi-layer ANN. Each node in the network can be considered a single-layered ANN (for simplicity, biases are not visualized in graphical model)</figcaption>

It turns out that the gradient information for the NN error surface can be calculated efficiently using a message passing algorithm known as the **backpropagation** algorithm:
- input signals are forward-propagated through the network toward the outputs, and 
- network errors are then calculated with respect to target variables and “backpropagated” backwards towards the inputs.

The <u>forward and backward signals</u> are then used to <u>determine the direction</u> in the parameter space to move <u>that lowers the network error</u>.

Figure 6 demonstrates the **4 key steps of the backpropagation** algorithm. The main concept: for a given observation we want to determine the degree of “responsibility” that each network parameter has for mis-predicting a target value associated with the observation. We then change that parameter according to this responsibility so that it reduces the network error.

To train an ANN, the 4 steps in Figure 6 are repeated iteratively by observing many input-target pairs and updating the parameters until:
- either the network error reaches a tolerably low value, 
- the parameters cease to update (convergence), or 
- a set number of iterations over the training data has been achieved.

## Neural Networks for Classification

<img align="right" style="width:200px;padding-left:25px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/truth-table.png">

Here we go over an example of training a single-layered neural network to perform a classification problem. The network is trained to learn a set of logical operators including the AND, OR, or XOR. To train the network we first generate training data. The inputs consist of 2-dimensional coordinates that span the input values $(x_1,x_2)$ values for a 2-bit truth table:

### `generate_classification_data` and `generate_regression_data`

We then perturb these observations by adding Normally-distributed noise. To generate target variables, we categorize each observations by applying one of logic operators (described in Figure 7) to the original (no-noisy) coordinates.

In [ ]:
def generate_classification_data(problem_type, n_obs_per_class=30):
    """Generates training data for all demos
    """
    np.random.seed(123)

    truth_table = np.array( [ [0, 0], [0, 1], [1, 0], [1, 1] ])
    ring_table = np.vstack( [ truth_table, np.array( [ [.5, .5], [1., .5], [0., .5], [.5, 0.], [.5, 1.] ]) ])
    ring_classes = [1., 1., 1., 1., 0., 1., 1., 1., 1.];

    problem_classes = {
        'AND': np.logical_and(truth_table[:,0], truth_table[:, 1]) * 1.,
        'OR':  np.logical_or( truth_table[:,0], truth_table[:, 1]) * 1.,
        'XOR': np.logical_xor(truth_table[:,0], truth_table[:, 1]) * 1.,
        'RING': ring_classes
    }

    if problem_type in ('AND', 'OR', 'XOR'):
        observations = np.tile(truth_table, (n_obs_per_class, 1)) + .15 * np.random.randn(n_obs_per_class * 4, 2)
        obs_classes = np.tile(problem_classes[problem_type], n_obs_per_class)
    else:
        observations = np.tile(ring_table, (n_obs_per_class, 1)) + .15 * np.random.randn(n_obs_per_class * 9, 2)
        obs_classes = np.tile(problem_classes[problem_type], n_obs_per_class)

    # Permute data
    permutation_idx = np.random.permutation(np.arange(len(obs_classes)))
    obs_classes = obs_classes[permutation_idx]
    observations = observations[permutation_idx]
    obs_x = [obs[0] for obs in observations]
    obs_y = [obs[1] for obs in observations]
    return obs_x, obs_y, obs_classes

Explanation of call to `generate_classification_data()`:

- observe final outputs

In [ ]:
problem_type='AND'
n_obs_per_class=3
obs_x, obs_y, obs_classes = generate_classification_data(problem_type=problem_type, 
                                                         n_obs_per_class=n_obs_per_class)
df = pd.DataFrame([problem_type, n_obs_per_class, int(len(obs_x)/n_obs_per_class)], 
                  index=["problem_type", "n_obs_per_class", "len(obs_x)/n_obs_per_class"])
display_htmlFcn2("df", "obs_x", "obs_y", 
                 "obs_classes", # its calculus explained in cells below
                 "np.rint(obs_x)","np.rint(obs_y)",
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[None,1,1,0])

- understand `problem_classes`

In [ ]:
truth_table = np.array( [ [0, 0], [0, 1], [1, 0], [1, 1] ])

ring_table = np.vstack( [ truth_table, np.array( [ [.5, .5], [1., .5], [0., .5], [.5, 0.], [.5, 1.] ]) ])
ring_classes = [1., 1., 1., 1., 0., 1., 1., 1., 1.]

problem_classes = {
    'AND': np.logical_and(truth_table[:,0], truth_table[:, 1]) * 1.,
    'OR':  np.logical_or( truth_table[:,0], truth_table[:, 1]) * 1.,
    'XOR': np.logical_xor(truth_table[:,0], truth_table[:, 1]) * 1.,
    'RING': ring_classes
}

# RING plot
plt.ioff() # prevent plots from being displayed in the output of Jupyter Notebook
fig, ax = plt.subplots()  
ax.scatter(*ring_table.T, c=ring_classes)
ax.tick_params(labelsize=7); ax.set_xticks(ax.get_xticks()[::2]); ax.set_yticks(ax.get_yticks()[::2])
plt.ion() # revert plt.ioff()
html_plot_fig = display_htmlFcn_plots(fig, width=190.0, height=200.0, caption="ax.scatter(*ring_table.T, c=ring_classes)")

display_htmlFcn2("truth_table", "problem_classes['AND']", "problem_classes['OR']","problem_classes['XOR']",
                 "ring_table","problem_classes['RING']", html_plot_fig,
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[0,0,0,0,1,0])

- how `obs_classes` (pre-permutated) is calculated:

In [ ]:
n_obs_per_class=3
problem_type = 'XOR'
problem_type_arr = problem_classes[problem_type]
observations = np.tile(truth_table, (n_obs_per_class, 1)) \
               + .15 * np.random.randn(n_obs_per_class * len(problem_type_arr), 2) # repeat input + add noise
obs_classes = np.tile(problem_type_arr, n_obs_per_class) # repeat target (no noisy)
display_htmlFcn2("problem_classes['XOR']", "observations", "obs_classes", 
                 "np.rint(observations)", # rint to notice how each |noise| of "observations" is <<<1
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[0,1,0])

- finally permutate vars to output

In [ ]:
# Permute data
permutation_idx = np.random.permutation(np.arange(len(obs_classes))) # np.random.permutation([1, 4, 9, 12, 15]) = array([15,  1,  9,  4, 12]) # random
obs_classes = obs_classes[permutation_idx]
observations = observations[permutation_idx]
obs_x = [obs[0] for obs in observations]
obs_y = [obs[1] for obs in observations]
display_htmlFcn2("permutation_idx","obs_x", "obs_y", "obs_classes", "np.rint(observations)",
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[0,1,1,0])

Explain `generate_regression_data()`:

In [ ]:
def generate_regression_data(problem_type='SIN', n_obs=100):
    np.random.seed(123)
    xx = np.linspace(-5, 5, n_obs);
    if problem_type == 'SIN':
        f = lambda x: 2.5 + np.sin(x)
    elif problem_type == 'ABS':
        f = lambda x: abs(x)

    yy = f(xx) + np.random.randn(*xx.shape)*.5
    perm_idx = np.random.permutation(np.arange(n_obs))
    return xx[perm_idx, None], yy[perm_idx]

In [ ]:
problem_type='SIN'
n_obs=10
np.random.seed(123)
xx = np.linspace(-5, 5, n_obs)

f = lambda x: 2.5 + np.sin(x)

yy = f(xx) + np.random.randn(*xx.shape)*.5 # sin(xx) + noise
perm_idx = np.random.permutation(np.arange(n_obs))

x, y = generate_regression_data(problem_type, n_obs=100) # call to generate_regression_data !!
p = np.poly1d(np.polyfit(x.flatten(), y.flatten(), 3))
t = np.linspace(min(x), max(x), 20)

plt.ioff() # prevent plots from being displayed in the output of Jupyter Notebook
fig, ax = plt.subplots()  
ax.plot(x, y, 'o', t, p(t), '-')
plt.ion() # revert plt.ioff()
html_plot_fig = display_htmlFcn_plots(fig, width=250.0, height=250.0, caption="ax.plot(x, y, 'o', t, p(t), '-')")


display_htmlFcn2("perm_idx", "xx[perm_idx, None]","yy[perm_idx]", html_plot_fig,
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[0,1,1,0])

### `initialize_network_parameters`

With training data in hand, we then train the network with the noisy inputs and binary categories targets using the gradient descent / backpropagation algorithm.

1st get from `generate_classification_data` the `observations` (inputs of ANN) and `targets` (desired outputs):

In [ ]:
# some args of run_ann_training_simulation
problem_type='AND'
n_observations=3

# start of run_ann_training_simulation
obs_x, obs_y, targets = generate_classification_data(problem_type, n_obs_per_class=n_observations)
observations = np.vstack([obs_x, obs_y]).T

print(f"problem_type: {problem_type}")
display_htmlFcn2("obs_x", "obs_y","targets", "np.rint(observations)", "observations",
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[1,1,0,0,1])

With the `observations` we can call `initialize_network_parameters`:

In [ ]:
def initialize_network_parameters(n_input_units, n_hidden_units=0, n_output_units=1):
    """Generate weights and bias parameters based on defined network architecture"""
    w1_size = n_hidden_units if n_hidden_units > 0 else n_output_units
    # Weights
    weights = dict()
    weight_gradients = dict()

    weights['w_1'] = np.random.rand(n_input_units, w1_size) - .5
    weight_gradients['w_1'] = np.zeros_like(weights['w_1'])

    if n_hidden_units > 0:
        weights['w_2'] = np.random.rand(n_hidden_units, n_output_units) - .5
        weight_gradients['w_2'] = np.zeros_like(weights['w_2'])

    # Biases
    biases = dict()
    bias_gradients = dict()
    biases['b_1'] = np.random.rand(w1_size) - .5
    bias_gradients['b_1'] = np.zeros_like(biases['b_1'])

    if n_hidden_units > 0:
        biases['b_2'] = np.random.rand(n_output_units) - .5
        bias_gradients['b_2'] = np.zeros_like(biases['b_2'])

    return weights, biases, weight_gradients, bias_gradients

In [ ]:
# some args of run_ann_training_simulation
n_hidden_units=0 # neurons in 1st hidden layer

# block of run_ann_training_simulation
# Initialize model parameters $\theta$
n_output_dims = 1
n_obs, n_input_dims = observations.shape
weights, biases, weight_gradients, bias_gradients = initialize_network_parameters(
    n_input_dims, n_hidden_units, n_output_dims
)

In [ ]:
weights, biases, weight_gradients, bias_gradients

In [ ]:
fig_NN_2_1 = plotNN01([2,1], weights=list(weights.values()), biases=list(biases.values()), figsize=(5,3));

In [ ]:
w = copy.deepcopy(weights)
w['w_1'][0][0] *= -1
display(w)
plotNN01([2,1], weights=list(w.values()), biases=list(biases.values()), figsize=(5,3),
         arrow_color_fcn=lambda x: "green" if x > 0 else "red",
         showVals=False, # to hide vals
         arrow_opacity_fcn=lambda x: .3,
         arrow_width = 0.001*50, arrow_width_variable = True);

In [ ]:
print(f"n_hidden_units: {n_hidden_units} (0 means no hidden layer, >0 are neurons in 1st hidden layer)")
for w,b in zip(weights.keys(),biases.keys()):
    display_htmlFcn2(f"weights['{w}']", f"biases['{b}']",f"weight_gradients['{w}']", f"bias_gradients['{b}']",
                     _localVars=locals(), _applyOrApplyMap=softZeros, _formatVal=["{:.3f}","{:.3f}","{:.1e}"])

Explanation of

```python
weights, biases, weight_gradients, bias_gradients = initialize_network_parameters(
    n_input_dims, n_hidden_units, n_output_dims
)
```

Remember:

```python
def initialize_network_parameters(n_input_units, n_hidden_units=0, n_output_units=1):
    ...
```

In [ ]:
# initialize_network_parameters
n_input_units = n_input_dims
print(f"n_input_units:  {n_input_units}")
print(f"n_hidden_units: {n_hidden_units} (0 means no hidden layer, >0 are neurons in 1st hidden layer)")
n_output_units = n_output_dims
print(f"n_output_units: {n_output_units}")

w1_size = n_hidden_units if n_hidden_units > 0 else n_output_units
print(f"w1_size: {w1_size}")

In [ ]:
# Weights
weights = dict()
weight_gradients = dict()

# n_cols dependent on w1_size (n_hidden_units if not 0, else n_output_units)
weights['w_1'] = np.random.rand(n_input_units, w1_size) - .5 # random uniform distrib. over [-.5, .5) = [0, 1) - .5
weight_gradients['w_1'] = np.zeros_like(weights['w_1']) # zeros

if n_hidden_units > 0: # False in this example
    weights['w_2'] = np.random.rand(n_hidden_units, n_output_units) - .5 # same
    weight_gradients['w_2'] = np.zeros_like(weights['w_2']) # same
    
for w in weights.keys():
    display_htmlFcn2(f"weights['{w}']", f"weight_gradients['{w}']",
                     _localVars=locals(), _applyOrApplyMap=softZeros, _formatVal=["{:.3f}","{:.1e}"])

In [ ]:
# Biases
biases = dict()
bias_gradients = dict()
biases['b_1'] = np.random.rand(w1_size) - .5 # no dependent on n_input_units as weights['w_1'] was
bias_gradients['b_1'] = np.zeros_like(biases['b_1'])

if n_hidden_units > 0: # False in this example
    biases['b_2'] = np.random.rand(n_output_units) - .5
    bias_gradients['b_2'] = np.zeros_like(biases['b_2'])

for w in biases.keys():
    display_htmlFcn2(f"biases['{w}']", f"bias_gradients['{w}']",
                     _localVars=locals(), _applyOrApplyMap=softZeros, _formatVal=["{:.3f}","{:.1e}"])

### `run_ann_training_simulation`

"ann" from Artificial Neural Network.

Next code block of `run_ann_training_simulation` (activation functions defined [here](#activation-function)):

In [ ]:
# Initialize problem-specific activation functions and their derivatives
g_activation = {}
g_activation_prime = {}
if problem_type in ('SIN', 'ABS'):  # regression using linear output (and optional tanh hidden) activations
    g_activation['g_out'], g_activation_prime['g_out'], _ = activation_functions['linear']
    if 'w_2' in weights:
        g_activation['g_1'], g_activation_prime['g_1'], _ = activation_functions['tanh']
else:  # classification using all sigmoid activations
    g_activation['g_out'], g_activation_prime['g_out'], _ = activation_functions['sigmoid']
    if 'w_2' in weights:
        g_activation['g_1'], g_activation_prime['g_1'], _ = activation_functions['sigmoid']

In [ ]:
import inspect 

def getLambdaFcn(fcn = "g_activation['g_out']"):
    print(f"{fcn:27}", end=" -->  ")
    print(inspect.getsource(eval(fcn)), end="")
    
getLambdaFcn()
getLambdaFcn(fcn = "g_activation_prime['g_out']")

Next code block is another setup, not used till end of function block `# Keep learning history for visualization` inside the `# Run the training`

In [ ]:
# global vars defined later when declared visualize_classification_learning()
PREDICTION_SURFACE_RESOLUTION = 20
PREDICTION_COLORMAP = 'spring'

In [ ]:
# Setup for learning history / visualization
loss_history = []
prediction_history = {}
weights_history = {}
biases_history = {}
if problem_type in ('SIN', 'ABS'):
    prediction_x = np.linspace(-5, 5, PREDICTION_SURFACE_RESOLUTION)
else:
    prediction_surface_range = np.linspace(-.5, 1.5, PREDICTION_SURFACE_RESOLUTION)
    prediction_surface_x, prediction_surface_y = np.meshgrid(prediction_surface_range, prediction_surface_range)
    prediction_surface_xy = [(x, y) for x, y in zip(prediction_surface_x.ravel(), prediction_surface_y.ravel())]

Summary of results before `# Run the training`:

In [ ]:
print(f"n_observations: {n_observations}")

print(f"problem_type: {problem_type}")

plt.ioff() # prevent plots from being displayed in the output of Jupyter Notebook
fig, ax = plt.subplots()  
ax.scatter(*observations.T, c=targets)
ax.tick_params(labelsize=7); aux=np.linspace(-.5,1.5,5); ax.set_xticks(aux); ax.set_yticks(aux)
ax.grid(False)
plt.ion() # revert plt.ioff()
html_plot_fig = display_htmlFcn_plots(fig, width=190.0, height=200.0, 
                                      caption="ax.scatter(*observations.T, c=targets)")

html_plot_fig2 = display_htmlFcn_plots(fig_NN_2_1,  width=150.0, height=200.0, 
                                       caption="Initial random weights & biases")
# remove grid-axis
id_fig = "fig2"
html_plot_fig2 = f'<style>#{id_fig} .mpld3-xaxis, #{id_fig} .mpld3-yaxis, #{id_fig}' \
+ ' .mpld3-grid { display: none; }</style>' + html_plot_fig2
html_plot_fig2 = f'<div style="display:inline;" id="{id_fig}">{html_plot_fig2}</div>'

display_htmlFcn2("obs_x", "obs_y","targets", "np.rint(observations)", "observations", 
                 html_plot_fig, html_plot_fig2,
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[1,1,0,0,1])

print(f"n_hidden_units: {n_hidden_units}")
for w,b in zip(weights.keys(),biases.keys()):
    display_htmlFcn2(f"weights['{w}']", f"biases['{b}']",f"weight_gradients['{w}']", f"bias_gradients['{b}']",
                     _localVars=locals(), _applyOrApplyMap=softZeros, _formatVal=["{:.3f}","{:.3f}","{:.1e}"])

getLambdaFcn()
getLambdaFcn(fcn = "g_activation_prime['g_out']")

#### Step I code

**Step I: forward-propagate input signal**

To forward-propagate the observed input forward through the network layers in order to provide a prediction for the current target $t$.

Note that in the Figure 6-I:
- $a_k$ could be considered network output (for a network with one hidden layer) or the output of a hidden layer that projects the remainder of the network (in the case of a network with more than one hidden layer). For this discussion, however, <u>we assume that the index $k$ is associated with the output layer</u> of the network, and thus each of the network outputs is designated by $a_k$
- when implementing this forward-propagation step, we should keep track of the feed-forward pre-activations $z_l$ and activations $a_l$ for all layers $l$, as these can be used to efficiently calculate backpropagated errors and error function gradients.

Sidenote: **Matrix Multiplication vs Element-wise Multiplication**.  

From [documentation](https://docs.python.org/3/reference/simple_stmts.html#augmented-assignment-statements):
> The `@` (at) operator is intended to be used for **matrix multiplication**.  [But not all (or maybe still none) builtin Python types implement this operator.]

From [stackoverflow](https://stackoverflow.com/a/30629255/9391770):
> `@` and `@=` are meant to clarify the confusion which existed so far with the operator `*` which was used either for element-wise multiplication or matrix multiplication depending on the convention employed in that particular library/code. As a result, in the future, the operator `*` is meant to be used for element-wise multiplication only. 

```
A = [[1, 2],    B = [[11, 12],
     [3, 4]]         [13, 14]]

A * B = [[1 * 11,   2 * 12], 
         [3 * 13,   4 * 14]]

A @ B  =  [[1 * 11 + 2 * 13,   1 * 12 + 2 * 14],
           [3 * 11 + 4 * 13,   3 * 12 + 4 * 14]]
```
           
**`@`** operator makes the code involving matrix multiplications much easier to read:

```python
# Current implementation of matrix multiplications using dot function
S = np.dot((np.dot(H, beta) - r).T,
            np.dot(inv(np.dot(np.dot(H, V), H.T)), np.dot(H, beta) - r))

# Current implementation of matrix multiplications using dot method
S = (H.dot(beta) - r).T.dot(inv(H.dot(V).dot(H.T))).dot(H.dot(beta) - r)

# Using the @ operator instead
S = (H @ beta - r).T @ inv(H @ V @ H.T) @ (H @ beta - r)
```

In [ ]:
def step_I_forwardprop(network_input, weights, biases, g_activation):
    if 'w_2' in weights:  # multi-layer network
        z_hidden = network_input @ weights['w_1'] + biases['b_1']
        a_hidden = g_activation['g_1'](z_hidden)
        z_output = a_hidden @ weights['w_2'] + biases['b_2']
    else:  # single-layer network
        z_hidden = np.array([])
        a_hidden = np.array([])
        z_output = network_input @ weights['w_1'] + biases['b_1']
    
    a_output = g_activation['g_out'](z_output) # Network prediction
    return a_output, z_output, a_hidden, z_hidden

<img align="right" style="width:400px;padding-left:45px;" src="https://dustinstansbury.github.io/theclevermachine/assets/images/a-gentle-introduction-to-neural-networks/multi-layer-perceptron.png">

$$
\begin{array}{llll}
a_{out}=a_k
&=g_k \bigg(b_k+\sum_j \bigg(g_j\big(b_j+\sum_i a_i w_{ij}\big)&\cdot w_{jk}\bigg)\bigg)\\
&=g_k \bigg(b_k+\sum_j \bigg(g_j\big(z_j                  \big)&\cdot w_{jk}\bigg)\bigg)\\
&=g_k \bigg(b_k+\sum_j \bigg(a_j                               &\cdot w_{jk}\bigg)\bigg)
\end{array}
$$

So,

$$
b_k+\sum_j \left(a_j \cdot w_{jk}\right)
= \underbrace{b_k}_{\text{biases['b_1']}}
+
\underbrace{\sum_j \left(a_j \cdot w_{jk}\right)}_{\text{network_input @ weights['w_1']}}
$$

equivalents in a single-layer network to

```python
z_output = network_input @ weights['w_1'] + biases['b_1']
```

And
$$
a_k
= g_k \left( b_k+\sum_j \left(a_j \cdot w_{jk}\right) \right)
= \underbrace{g_k}_{\text{ g_activation['g_out'] }}
\bigg(
\underbrace{ b_k+\sum_j \left(a_j \cdot w_{jk} \right)}_{\text{z_output}}
\bigg)
$$

equivalents

```python
a_output = g_activation['g_out'](z_output) # Network prediction
```




- In our case with `n_hidden_units = 0` (no hidden layer, just 2 inputs and 1 output (and 1 target) for each observation) we had (with $n$ observations) next. Each row of $\vec{a_k}$ is result of each input (`network_input`) of next loop explained at end of present subsection

```python
for i, (network_input, target) in enumerate(zip(observations, targets)): 
    ...
    a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(network_input, weights, biases, g_activation)
    ...
```

In [ ]:
plotNN01([2,1], figsize=(3,2), showVals=False);

$$
\vec{a_k}
\,=\,
\overrightarrow{
g_k \left( b_k+\sum_j \left(a_j \cdot w_{jk}\right) \right)
}
\,=\,
g_k
\left(
    \left[
    \begin{array}{ccc}
        a^{(0)}_{1_{1}}
        &a^{(0)}_{2_{1}}
        &1
        \\
        a^{(0)}_{1_{2}}
        &a^{(0)}_{2_{2}}
        &1
        \\
        \vdots
        &\vdots
        &\vdots
        \\
        a^{(0)}_{1_{n}}
        &a^{(0)}_{2_{n}}
        &1
    \end{array}
    \right]
    \,\times\,
    \left[
    \begin{array}{cccc}
        \omega_{1-1}^{(1)}
        \\
        \omega_{1-2}^{(1)}
        \\
        b_1^{(1)}
    \end{array}
    \right]        
\right)
\,=\,
g_k
\left(
    \left[
    \begin{array}{cccc}
     a_{1_{1}}^{(0)}\omega_{1-1}^{(1)} \,+\, a_{2_{1}}^{(0)}\omega_{1-2}^{(1)} \,+\, b_1^{(1)}
    \\
     a_{1_{2}}^{(0)}\omega_{1-1}^{(1)} \,+\, a_{2_{2}}^{(0)}\omega_{1-2}^{(1)} \,+\, b_1^{(1)}
    \\
     \vdots
    \\
     a_{1_{n}}^{(0)}\omega_{1-1}^{(1)} \,+\, a_{2_{n}}^{(0)}\omega_{1-2}^{(1)} \,+\, b_1^{(1)}
    \end{array}
    \right]        
\right)
\,=\,
g_k
\left(
    \left[
    \begin{array}{cccc}
     z_{1_{1}}^{(1)}
    \\
     z_{1_{2}}^{(1)}
    \\
     \vdots
    \\
     z_{1_{n}}^{(1)}
    \end{array}
    \right]        
\right)
\,=\,
    \left[
    \begin{array}{cccc}
     a_{1_{1}}^{(1)}
    \\
     a_{1_{2}}^{(1)}
    \\
     \vdots
    \\
     a_{1_{n}}^{(1)}
    \end{array}
    \right]
$$

- with `n_hidden_units = 1` and that hidden layer with 4 cells, then we had (for example with $n$ observations of 2 cells each $(a^{(0)}_{1_{1}},a^{(0)}_{2_{1}})$ to $(a^{(0)}_{1_{n}},a^{(0)}_{2_{n}})$):
    - 1st calculate $\vec{z_j}$ and $\vec{a_j}$ ($z$ and $output$ of hidden layer 1 (one and only hidden layer here)
        ```python
        z_hidden = network_input @ weights['w_1'] + biases['b_1']
        a_hidden = g_activation['g_1'](z_hidden)
        ```

In [ ]:
plotNN01_2_4_1 = plotNN01([2,4,1], figsize=(8,3), showVals=False, radialBool=0);

$$
\begin{array}{ll}
&\vec{a_j}
\,=\,
\overrightarrow{
g_j \left( b_j+\sum_i \left(a_i \cdot w_{ij}\right) \right)
}
\,=\,
g_j
\left(
    \left[
    \begin{array}{ccc}
        a^{(0)}_{1_{1}}
        &a^{(0)}_{2_{1}}
        &1
        \\
        a^{(0)}_{1_{2}}
        &a^{(0)}_{2_{2}}
        &1
        \\
        \vdots
        &\vdots
        &\vdots
        \\
        a^{(0)}_{1_{n}}
        &a^{(0)}_{2_{n}}
        &1
    \end{array}
    \right]
    \,\times\,
    \left[
    \begin{array}{cccc}
        \omega_{1-1}^{(1)}
        &\omega_{2-1}^{(1)}
        &\omega_{3-1}^{(1)}
        &\omega_{4-1}^{(1)}
        \\
        \omega_{1-2}^{(1)}
        &\omega_{2-2}^{(1)}
        &\omega_{3-2}^{(1)}
        &\omega_{4-2}^{(1)}
        \\
        b_1^{(1)}
        &b_2^{(1)}
        &b_3^{(1)}
        &b_4^{(1)}
    \end{array}
    \right]        
\right)
\\
&\,=\,
g_j
\left(
    \left[
    \begin{array}{cccc}
     a_{1_{1}}^{(0)}\omega_{1-1}^{(1)} \,+\, a_{2_{1}}^{(0)}\omega_{1-2}^{(1)} \,+\, b_1^{(1)} 
    &a_{1_{1}}^{(0)}\omega_{2-1}^{(1)} \,+\, a_{2_{1}}^{(0)}\omega_{2-2}^{(1)} \,+\, b_2^{(1)}
    &a_{1_{1}}^{(0)}\omega_{3-1}^{(1)} \,+\, a_{2_{1}}^{(0)}\omega_{3-2}^{(1)} \,+\, b_3^{(1)}
    &a_{1_{1}}^{(0)}\omega_{4-1}^{(1)} \,+\, a_{2_{1}}^{(0)}\omega_{4-2}^{(1)} \,+\, b_4^{(1)}
    \\
     a_{1_{2}}^{(0)}\omega_{1-1}^{(1)} \,+\, a_{2_{2}}^{(0)}\omega_{1-2}^{(1)} \,+\, b_1^{(1)} 
    &a_{1_{2}}^{(0)}\omega_{2-1}^{(1)} \,+\, a_{2_{2}}^{(0)}\omega_{2-2}^{(1)} \,+\, b_2^{(1)}
    &a_{1_{2}}^{(0)}\omega_{3-1}^{(1)} \,+\, a_{2_{2}}^{(0)}\omega_{3-2}^{(1)} \,+\, b_3^{(1)}
    &a_{1_{2}}^{(0)}\omega_{4-1}^{(1)} \,+\, a_{2_{2}}^{(0)}\omega_{4-2}^{(1)} \,+\, b_4^{(1)}
    \\
     \vdots 
    &\vdots
    &\vdots
    &\vdots
    \\
     a_{1_{n}}^{(0)}\omega_{1-1}^{(1)} \,+\, a_{2_{n}}^{(0)}\omega_{1-2}^{(1)} \,+\, b_1^{(1)} 
    &a_{1_{n}}^{(0)}\omega_{2-1}^{(1)} \,+\, a_{2_{n}}^{(0)}\omega_{2-2}^{(1)} \,+\, b_2^{(1)}
    &a_{1_{n}}^{(0)}\omega_{3-1}^{(1)} \,+\, a_{2_{n}}^{(0)}\omega_{3-2}^{(1)} \,+\, b_3^{(1)}
    &a_{1_{n}}^{(0)}\omega_{4-1}^{(1)} \,+\, a_{2_{n}}^{(0)}\omega_{4-2}^{(1)} \,+\, b_4^{(1)}
    \end{array}
    \right]        
\right)
\\
&\,=\,
g_j
\left(
    \left[
    \begin{array}{cccc}
     z_{1_{1}}^{(1)} 
    &z_{2_{1}}^{(1)}
    &z_{3_{1}}^{(1)}
    &z_{4_{1}}^{(1)}
    \\
     z_{1_{2}}^{(1)} 
    &z_{2_{2}}^{(1)}
    &z_{3_{2}}^{(1)}
    &z_{4_{2}}^{(1)}
    \\
     \vdots 
    &\vdots
    &\vdots
    &\vdots
    \\
     z_{1_{n}}^{(1)} 
    &z_{2_{n}}^{(1)}
    &z_{3_{n}}^{(1)}
    &z_{4_{n}}^{(1)}
    \end{array}
    \right]        
\right)
\,=\,
    \left[
    \begin{array}{cccc}
     a_{1_{1}}^{(1)} 
    &a_{2_{1}}^{(1)}
    &a_{3_{1}}^{(1)}
    &a_{4_{1}}^{(1)}
    \\
     a_{1_{2}}^{(1)} 
    &a_{2_{2}}^{(1)}
    &a_{3_{2}}^{(1)}
    &a_{4_{2}}^{(1)}
    \\
     \vdots 
    &\vdots
    &\vdots
    &\vdots
    \\
     a_{1_{n}}^{(1)} 
    &a_{2_{n}}^{(1)}
    &a_{3_{n}}^{(1)}
    &a_{4_{n}}^{(1)}
    \end{array}
    \right]        
\end{array}
$$

Note this LaTeX notation subindixes refere first to target layer then to origin layer, so for example $\omega_{4-2}^{(1)}$ actually represents $\omega_{i_2j_4}$ of Python plot.

- with `n_hidden_units = 4` (4 neurons in unique hidden layer) ...
    - 2nd calculate $\vec{z_k}$ and $\vec{a_k}$
    ```python
    z_output = a_hidden @ weights['w_2'] + biases['b_2']
    ```

$
\begin{array}{ll}
\vec{a_k}
\,=\,
\overrightarrow{
g_k \left( b_k+\sum_j \left(a_j \cdot w_{jk}\right) \right)
}
\,=\,
g_k
\left(
    \left[
    \begin{array}{cccc}
         a_{1_{1}}^{(1)} 
        &a_{2_{1}}^{(1)}
        &a_{3_{1}}^{(1)}
        &a_{4_{1}}^{(1)}
        &1
        \\
         a_{1_{2}}^{(1)} 
        &a_{2_{2}}^{(1)}
        &a_{3_{2}}^{(1)}
        &a_{4_{2}}^{(1)}
        &1
        \\
         \vdots 
        &\vdots 
        &\vdots 
        &\vdots 
        &\vdots 
        \\
         a_{1_{n}}^{(1)} 
        &a_{2_{n}}^{(1)}
        &a_{3_{n}}^{(1)}
        &a_{4_{n}}^{(1)}
        &1
    \end{array}
    \right] 
    \,\cdot\,
    \left[
    \begin{array}{cccc}
        \omega_{1-1}^{(2)} \\
        \omega_{1-2}^{(2)} \\
        \omega_{1-3}^{(2)} \\
        \omega_{1-4}^{(2)} \\
        b_1^{(2)}
    \end{array}
    \right]
\right)
\,=\,
g_k
\left(
    \left[
    \begin{array}{cccc}
             a_{1_{1}}^{(1)} \omega_{1-1}^{(2)} 
        \,+\,a_{2_{1}}^{(1)} \omega_{1-2}^{(2)}
        \,+\,a_{3_{1}}^{(1)} \omega_{1-3}^{(2)}
        \,+\,a_{4_{1}}^{(1)} \omega_{1-4}^{(2)}
        \,+\,b_1^{(2)}
        \\
             a_{1_{2}}^{(1)} \omega_{1-1}^{(2)} 
        \,+\,a_{2_{2}}^{(1)} \omega_{1-2}^{(2)}
        \,+\,a_{3_{2}}^{(1)} \omega_{1-3}^{(2)}
        \,+\,a_{4_{2}}^{(1)} \omega_{1-4}^{(2)}
        \,+\,b_1^{(2)}
        \\
         \vdots 
        \\
             a_{1_{n}}^{(1)} \omega_{1-1}^{(2)}
        \,+\,a_{2_{n}}^{(1)} \omega_{1-2}^{(2)}
        \,+\,a_{3_{n}}^{(1)} \omega_{1-3}^{(2)}
        \,+\,a_{4_{n}}^{(1)} \omega_{1-4}^{(2)}
        \,+\,b_1^{(2)}
    \end{array}
    \right]
\right)
\,=\,
g_k
\left(
    \left[
    \begin{array}{cccc}
     z_{1_{1}}^{(2)} \\
     z_{1_{2}}^{(2)} \\
     \vdots  \\
     z_{1_{n}}^{(2)} 
    \end{array}
    \right]        
\right)
\,=\,
\left[
    \begin{array}{cccc}
     a_{1_{1}}^{(2)} \\
     a_{1_{2}}^{(2)} \\
     \vdots  \\
     a_{1_{n}}^{(2)} 
    \end{array}
\right]
\end{array}
$

-------------
As we will see in [#Whole-function](#Whole-function) section, the `run_ann_training_simulation` loops `n_iterations`, and enclosed the loop of each `(network_input, target)`:

In [ ]:
# some args of run_ann_training_simulation
n_iterations=2
learning_rate=3

In [ ]:
for iteration in range(n_iterations):
    obs_error = []
    print(f"iteration: {iteration}")
    for i, (network_input, target) in enumerate(zip(observations, targets)):
        network_input = np.atleast_2d(network_input)
        
        # Step I: Forward propagate input signal through the network,
        # collecting activations and hidden states
        a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(
            network_input, weights, biases, g_activation
        )
        
        if i==iteration: # random
            print(f"i: {i}")
            display_htmlFcn2("network_input", "weights['w_1']",
                 "network_input @ weights['w_1']",
                 "biases['b_1']", "network_input @ weights['w_1'] + biases['b_1']",
                 "g_activation['g_out'](z_output)", # a_output
                 "(lambda z: 1./(1. + np.exp(-z)))(network_input @ weights['w_1'] + biases['b_1'])", # a_output
                 _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=2)        

#### Step II code

**Step II: back-propagate error signals**

Calculate the network output error and backpropagate it toward the input. For this walkthrough we’ll continue to use the sum of squared differences error function, this time written in a more explicit form than in the [Training neural networks & gradient descent](#Training-neural-networks-&-gradient-descent) section:

$$
\begin{align}E(\mathbf{\theta}) &= \frac{1}{2}\sum_{k \in K}(a_k - t_k)^2\end{align}
$$

Here we sum over the values of all $k$ output units (one in this example). Note that the model parameters parameters $\theta$ are implicit in the output activations $a_k$. This error function has the following derivative with respect to the model parameters $\theta$:

$$
E'(\mathbf{\theta}) = (a_k - t_k)
$$

We can now define an “error signal” $\theta_k$ at the **output node** that will be backpropagated toward the input. The error signal is calculated as follows:

$$
\begin{align}
\delta_k &= g_k'(z_k)E'(\theta) \\  
&= g_k'(z_k)(a_k - t_k)
\end{align}
$$

This error signal essentially weights the gradient of the error function by the gradient of the output activation function. Notice that there is a $z_k$ term is used in the calculation of $\delta_k$. In order to make learning more efficient, we keep track of the $z_k$ during the forward-propagation step so that it can be used in backpropagation, notice `z_output` as 2nd output of:
```python
a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(network_input, weights, biases, g_activation)
```

We can continue backpropagating the error signal toward the input by passing $\delta_k$ through the output layer weights $w_{jk}$, summing over all output nodes, and passing the result through the gradient of the activation function at the **hidden layer** $g_j'(z_j)$ (Figure 6-II). Performing these operations results in the backpropagated error signal for the hidden layer, $\delta_j$:


$$
\delta_j = g_j'(z_j)\sum_k \delta_k w_{jk}
$$

For networks that have more than one hidden layer, this error backpropagation procedure can continue for layers $j−1$,$j−2$,..., etc.

Summary:

- output layer $k$ gives `delta_output`:
$$
\delta_k  = g_k'(z_k)(a_k - t_k)
$$

- hidden layer $j$ and direct forward layer $k$ gives `delta_hidden` (of 1st hidden layer):
$$
\delta_j = g_j'(z_j)\sum_k \delta_k w_{jk}
$$

In [ ]:
def step_II_backprop(target, a_output, z_output, z_hidden, weights, g_activation_prime):
    # Calculate error function derivative given input/output/params
    delta_output = g_activation_prime['g_out'](z_output) * (a_output - target)

    # Calculate any error contributions from hidden layers nodes
    if 'w_2' in weights:  # multi-layer network
        delta_hidden = g_activation_prime['g_1'](z_hidden) * (delta_output @ weights['w_2'].T)
    else:
        delta_hidden = np.array([])
    return delta_output, delta_hidden

Next is for $n$ observations and just 1 output neuron. 
Each row of $\vec{\delta_k}$ is result of each input-target (`(network_input, target)`) of next loop explained at end of present subsection

```python
for i, (network_input, target) in enumerate(zip(observations, targets)): 
    ...
    a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(network_input, weights, biases, g_activation)
    delta_output, delta_hidden = step_II_backprop(target, a_output, z_output, z_hidden, weights, g_activation_prime)
    ...
```

$$
\vec{\delta_k}
\,=\, \overrightarrow{error} \cdot \overrightarrow{g_k'(z_k)}
\,=\,
    \left(
        \left[
        \begin{array}{ccc}
            t_1\\
            t_2\\
            \vdots\\
            t_n\\
        \end{array}
        \right] 
        \,-\,
        \left[
        \begin{array}{cccc}
         a_{1_{1}}^{(l)} 
        \\
         a_{1_{2}}^{(l)} 
        \\
         \vdots
        \\
         a_{1_{n}}^{(l)} 
        \end{array}
        \right]
    \right)
    \,\cdot\,
    g_k'
    \left(
        \left[
        \begin{array}{cccc}
         z_{1_{1}}^{(l)} 
        \\
         z_{1_{2}}^{(l)} 
        \\
         \vdots 
        \\
         z_{1_{n}}^{(l)} 
        \end{array}
        \right]
    \right)
\,=\,
\left[
\begin{array}{cccc}
    (t_1 - a_{1_{1}}^{(l)}) \cdot g_k'\left(z_{1_{1}}^{(l)}\right)\\
    (t_2 - a_{1_{2}}^{(l)}) \cdot g_k'\left(z_{1_{2}}^{(l)}\right)\\
    \vdots \\
    (t_n - a_{1_{n}}^{(l)}) \cdot g_k'\left(z_{1_{n}}^{(l)}\right)
\end{array}
\right]
$$

Replace each variable of style $x^{(l)}$ with $x^{(1)}$ if NN has no hidden layer ($l$ from layer), or with $x^{(2)}$ if 1 hidden layer.

In Python

$$
\delta_k  = 
\underbrace{
g_k'(z_k)
}_{\text{g_activation_prime['g_out'](z_output)}}
\underbrace{
(a_k - t_k)
}_{\text{(a_output - target)}}
$$

$\bullet$ With `n_hidden_units = 0` (no hidden layer), $\vec{\delta_j}$ does not exist:

$$
\delta_j = 
\underbrace{
g_j'(z_j)\sum_k \delta_k w_{jk}
}_{\text{np.array([])}}
$$

$\bullet$ If the hidden layer exists (with `n_hidden_units`=4 neurons), then:

In [ ]:
plotNN01_2_4_1

$$
\vec{\delta_j} = 
\overrightarrow{
g_j'(z_j)\sum_k \delta_k w_{jk}
}
=
g_j'
\left(
    \left[
    \begin{array}{cccc}
      z_{1_{1}}^{(1)}
     &z_{2_{1}}^{(1)}
     &z_{3_{1}}^{(1)}
     &z_{4_{1}}^{(1)}\\
      z_{1_{2}}^{(1)}
     &z_{2_{2}}^{(1)}
     &z_{3_{2}}^{(1)}
     &z_{4_{2}}^{(1)}\\
      \vdots
     &\vdots
     &\vdots
     &\vdots\\
      z_{1_{n}}^{(1)}
     &z_{2_{n}}^{(1)}
     &z_{3_{n}}^{(1)}
     &z_{4_{n}}^{(1)}
    \end{array}
    \right]
\right)
\cdot
\left[
\begin{array}{cccc}
    (t_1 - a_{1_{1}}^{(2)}) \cdot g_k'\left(z_{1_{1}}^{(2)}\right)\\
    (t_2 - a_{1_{2}}^{(2)}) \cdot g_k'\left(z_{1_{2}}^{(2)}\right)\\
    \vdots \\
    (t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)
\end{array}
\right]
\times
\left[
\begin{array}{cccc}
    \omega_{1-1}^{(2)}\\
    \omega_{1-2}^{(2)}\\
    \omega_{1-3}^{(2)}\\
    \omega_{1-4}^{(2)}
\end{array}
\right]^T
=
g_j'
\left(
    \left[
    \begin{array}{cccc}
      z_{1_{1}}^{(1)}
     &z_{2_{1}}^{(1)}
     &z_{3_{1}}^{(1)}
     &z_{4_{1}}^{(1)}\\
      z_{1_{2}}^{(1)}
     &z_{2_{2}}^{(1)}
     &z_{3_{2}}^{(1)}
     &z_{4_{2}}^{(1)}\\
      \vdots
     &\vdots
     &\vdots
     &\vdots\\
      z_{1_{n}}^{(1)}
     &z_{2_{n}}^{(1)}
     &z_{3_{n}}^{(1)}
     &z_{4_{n}}^{(1)}
    \end{array}
    \right]
\right)
\cdot
\left[
\begin{array}{cccc}
  (t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-1}^{(2)}
 &(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-2}^{(2)}
 &(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-3}^{(2)}
 &(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-4}^{(2)}\\
  (t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-1}^{(2)}
 &(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-2}^{(2)}
 &(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-3}^{(2)}
 &(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-4}^{(2)}\\
  \vdots
 &\vdots
 &\vdots
 &\vdots\\
  (t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}
 &(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-2}^{(2)}
 &(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-3}^{(2)}
 &(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-4}^{(2)}\\
\end{array}
\right]
=
\left[
\begin{array}{cccc}
  g_j'(z_{1_{1}}^{(1)})(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-1}^{(2)}
 &g_j'(z_{2_{1}}^{(1)})(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-2}^{(2)}
 &g_j'(z_{3_{1}}^{(1)})(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-3}^{(2)}
 &g_j'(z_{4_{1}}^{(1)})(t_1 - a_{1_{1}}^{(2)})\cdot g_k'(z_{1_{1}}^{(2)})\cdot\omega_{1-4}^{(2)}\\
  g_j'(z_{1_{2}}^{(1)})(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-1}^{(2)}
 &g_j'(z_{2_{2}}^{(1)})(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-2}^{(2)}
 &g_j'(z_{3_{2}}^{(1)})(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-3}^{(2)}
 &g_j'(z_{4_{2}}^{(1)})(t_2 - a_{1_{2}}^{(2)})\cdot g_k'(z_{1_{2}}^{(2)})\cdot\omega_{1-4}^{(2)}\\
  \vdots
 &\vdots
 &\vdots
 &\vdots\\
  g_j'(z_{1_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}
 &g_j'(z_{2_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-2}^{(2)}
 &g_j'(z_{3_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-3}^{(2)}
 &g_j'(z_{4_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-4}^{(2)}\\
\end{array}
\right]
$$

Notice that for just 1 output neuron $\vec{\delta_k}$ is 1D array, thus the delta of each neuron $j$ of unique hidden layer can be simplified to:
$$
\vec{\delta_j}
= 
\overrightarrow{
g_j'(z_j)\sum_k \delta_k w_{jk}
}
= 
\overrightarrow{
g_j'(z_j)\delta_k w_{jk}
}
$$

for example neuron $j_{1}$:
$\vec{\delta_{j_{1}}}=g_j'(z_{j_{1}})\delta_k w_{j_{1}k_{1}}$
which values in $n$-th observation using matrix-notation 
$g_j'(z_{1_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}$

If we had 2 output neurons then $\vec{\delta_k}$ and therefore calculus of $\vec{\delta_j}$ were harder.

$$
\vec{\delta_k}_{\text{2 neurons}}
=
\left[
\begin{array}{cccc}
     (t_{1_{1}} - a_{1_{1}}^{(2)}) \cdot g_k'\left(z_{1_{1}}^{(2)}\right)
    &(t_{2_{1}} - a_{2_{1}}^{(2)}) \cdot g_k'\left(z_{2_{1}}^{(2)}\right)\\
     (t_{1_{2}} - a_{1_{2}}^{(2)}) \cdot g_k'\left(z_{1_{2}}^{(2)}\right)
    &(t_{2_{2}} - a_{2_{2}}^{(2)}) \cdot g_k'\left(z_{2_{2}}^{(2)}\right)\\
    \vdots 
    &\vdots\\    
     (t_{1_{n}} - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)
    &(t_{2_{n}} - a_{2_{n}}^{(2)}) \cdot g_k'\left(z_{2_{n}}^{(2)}\right)
\end{array}
\right]
$$

In [ ]:
for iteration in range(n_iterations):
    obs_error = []
    print(f"iteration: {iteration}")
    for i, (network_input, target) in enumerate(zip(observations, targets)):
        network_input = np.atleast_2d(network_input)
        
        # Step I: Forward propagate input signal through the network,
        # collecting activations and hidden states
        a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(
            network_input, weights, biases, g_activation
        )
        
        # Step II: Backpropagate error signal
        delta_output, delta_hidden = step_II_backprop(
            target, a_output, z_output, z_hidden, weights, g_activation_prime
        )
        if i==iteration: # random
            print(f"i: {i}")
            display_htmlFcn2("a_output", "[target]", 
                             "a_output - target",
                             "z_output",
                             "g_activation_prime['g_out'](z_output)",
                             "g_activation_prime['g_out'](z_output) * (a_output - target)", # delta_output
                             _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=[2,0,2])

#### Step III code


**Step III: calculate parameter gradients**

Calculate the gradients of the error function with respect to the model parameters at each layer $l$ using the forward signals $a_{l−1}$, and the backward error signals $\delta_l$ . If one considers the model weights $w_{l−1,l}$ at a layer $l$ as linking the forward signal $a_{l−1}$ to the error signal $\delta_l$ (Figure 6-III), then the gradient of the error function with respect to those weights is:

$$
\frac{\partial E}{\partial w_{l-1, l}} = a_{l-1}\delta_l
$$

Thus the gradient of the error function with respect to the model weight at each layer can be efficiently calculated by simply keeping track of the forward-propagated activations feeding into that layer from below, and weighting those activations by the backward-propagated error signals feeding into that layer from above!

What about the bias parameters? It turns out that the same gradient rule used for the weights applies, except that “feed-forward activations” for biases are always +1 (see Figure 1). Thus the bias gradients for layer $l$
are simply:

$$
\frac{\partial E}{\partial b_{l}} = (1)\delta_l = \delta_l
$$

Summary:

$$
\frac{\partial E}{\partial w_{l-1, l}} = a_{l-1}\delta_l
$$

$$
\frac{\partial E}{\partial b_{l}} = (1)\delta_l = \delta_l
$$

$\bullet$ With `n_hidden_units = 0`. For $n$ observations. 
Each row of $\overrightarrow{
\frac{\partial E}{\partial w_{l-1, l}} }$ and $\overrightarrow{\frac{\partial E}{\partial b_{l}}}$ is result of each input-target (`(network_input, target)`) of next loop explained at end of present subsection

```python
for i, (network_input, target) in enumerate(zip(observations, targets)): 
    ...
    a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(network_input, weights, biases, g_activation)
    delta_output, delta_hidden = step_II_backprop(target, a_output, z_output, z_hidden, weights, g_activation_prime)
    weight_gradients, bias_gradients = step_III_gradient_calculation(delta_output, delta_hidden, a_hidden, network_input, weight_gradients, bias_gradients)
    ...
```

$$
\overrightarrow{
\frac{\partial E}{\partial w_{l-1, l}} 
}
= \overrightarrow{
\frac{\partial E}{\partial w_{j k}}
}
= \vec{a_{j}}\cdot\vec{\delta_k}
=
\left[
\begin{array}{cccc}
      a^{(0)}_{1_{1}} 
     &a^{(0)}_{2_{1}}\\
      a^{(0)}_{1_{2}}  
     &a^{(0)}_{2_{2}}\\
      \vdots
     &\vdots\\
      a^{(0)}_{1_{n}} 
     &a^{(0)}_{2_{n}} 
\end{array}
\right]^T
\cdot
\vec{\delta_k}
$$

$$
\overrightarrow{
\frac{\partial E}{\partial b_{l}}
}
=
\overrightarrow{
\frac{\partial E}{\partial b_{k}}
}
= \vec{\delta_k}
$$

where $\vec{\delta_k}$ was calculated in step II.

$\bullet$ With `n_hidden_units = 0` (no hidden layer). For example weight and bias gradients of of obsevation $n$ (no hidden layer and just 1 ouput):

$$
\overrightarrow{
\frac{\partial E}{\partial w_{{l-1, l}_{n}}} 
}
= \overrightarrow{
\frac{\partial E}{\partial w_{{j k}_{n}}}
}
=
\left[
\begin{array}{cccc}
     \frac{\partial E}{\partial w_{j_{1}k_{1}}}_{n}\\
     \frac{\partial E}{\partial w_{j_{2}k_{1}}}_{n} 
\end{array}
\right]
= \vec{a_{j_{n}}}\cdot\vec{\delta_{k_{n}}}
=
\underbrace{
    \left[
    \begin{array}{cccc}
          a^{(0)}_{1_{n}}
         &a^{(0)}_{2_{n}} 
    \end{array}
    \right]^T
}_{\text{network_input.T}}
\cdot
\underbrace{
    \left[
    (t_n - a_{1_{n}}^{(1)}) \cdot g_k'\left(z_{1_{n}}^{(1)}\right)
    \right]
}_{\text{delta_output}}
=
\left[
\begin{array}{cccc}
      a^{(0)}_{1_{n}}(t_n - a_{1_{n}}^{(1)}) \cdot g_k'\left(z_{1_{n}}^{(1)}\right)\\
      a^{(0)}_{2_{n}}(t_n - a_{1_{n}}^{(1)}) \cdot g_k'\left(z_{1_{n}}^{(1)}\right)
\end{array}
\right]
$$

$$
\overrightarrow{
\frac{\partial E}{\partial b_{l_{n}}}
}
=
\overrightarrow{
\frac{\partial E}{\partial b_{k_{n}}}
}
=
\left[
\begin{array}{cccc}
     \frac{\partial E}{\partial b_{k1}}_{n} 
\end{array}
\right]
= \vec{\delta_{k_{n}}}
=
\left[
\begin{array}{cccc}
      (t_n - a_{1_{n}}^{(1)}) \cdot g_k'\left(z_{1_{n}}^{(1)}\right)
\end{array}
\right]
$$

$\bullet$ If the hidden layer exists (with `n_hidden_units`=4 neurons), then $\vec{\delta_k}$ and $\vec{\delta_j}$ are as complex as showed in step II. Thus,
 - the biases gradients are those delta arrays, while 
 - the weights gradients are those $\delta$ multiplied respect. by $\vec{a_k}$ or $\vec{a_j}$.

In [ ]:
plotNN01_2_4_1

$$
\underbrace{
\overrightarrow{
\frac{\partial E}{\partial w_{{j k}_{n}}}
}
}_{\text{weight_gradients['w_2']}}
=
\left[
\begin{array}{cccc}
     \frac{\partial E}{\partial w_{j_{1}k_{1}}}_{n}\\
     \frac{\partial E}{\partial w_{j_{2}k_{1}}}_{n}\\
     \frac{\partial E}{\partial w_{j_{3}k_{1}}}_{n}\\
     \frac{\partial E}{\partial w_{j_{4}k_{1}}}_{n}  
\end{array}
\right]
= \vec{a_{j_{n}}}\cdot\vec{\delta_{k_{n}}}
=
\underbrace{
    \left[
    \begin{array}{cccc}
          a^{(1)}_{1_{n}}
         &a^{(1)}_{2_{n}} 
         &a^{(1)}_{3_{n}} 
         &a^{(1)}_{4_{n}} 
    \end{array}
    \right]^T
}_{\text{a_hidden.T}}
\cdot
\underbrace{
    \left[
    (t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)
    \right]
}_{\text{delta_output}}
=
\left[
\begin{array}{cccc}
      a^{(1)}_{1_{n}}(t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)\\
      a^{(1)}_{2_{n}}(t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)\\
      a^{(1)}_{3_{n}}(t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)\\
      a^{(1)}_{4_{n}}(t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)
\end{array}
\right]
$$

$$
\underbrace{
\overrightarrow{
\frac{\partial E}{\partial b_{k_{n}}}
}
}_{\text{bias_gradients['b_2']}}
=
\left[
\begin{array}{cccc}
     \frac{\partial E}{\partial b_{k1}}_{n} 
\end{array}
\right]
= \vec{\delta_{k_{n}}}
=
\underbrace{
    \left[
    (t_n - a_{1_{n}}^{(2)}) \cdot g_k'\left(z_{1_{n}}^{(2)}\right)
    \right]
}_{\text{delta_output}}
$$

$$
\underbrace{
\overrightarrow{
\frac{\partial E}{\partial w_{{i j}_{n}}}
}
}_{\text{weight_gradients['w_1']}}
=
\left[
\begin{array}{cccc}
      \frac{\partial E}{\partial w_{i_{1}j_{1}}}_{n}
     &\frac{\partial E}{\partial w_{i_{1}j_{2}}}_{n}
     &\frac{\partial E}{\partial w_{i_{1}j_{3}}}_{n}
     &\frac{\partial E}{\partial w_{i_{1}j_{4}}}_{n}\\
      \frac{\partial E}{\partial w_{i_{2}j_{1}}}_{n}
     &\frac{\partial E}{\partial w_{i_{2}j_{2}}}_{n}
     &\frac{\partial E}{\partial w_{i_{2}j_{3}}}_{n}
     &\frac{\partial E}{\partial w_{i_{2}j_{4}}}_{n}
\end{array}
\right]
= \vec{a_{i_{n}}}\cdot\vec{\delta_{j_{n}}}
=
\underbrace{
    \left[
    \begin{array}{cccc}
          a^{(0)}_{1_{n}}
         &a^{(0)}_{2_{n}} 
    \end{array}
    \right]^T
}_{\text{network_input.T}}
\cdot
\underbrace{
    \left[
    \begin{array}{cccc}
       g_j'(z_{1_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}\\
       g_j'(z_{2_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-2}^{(2)}\\
       g_j'(z_{3_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-3}^{(2)}\\
       g_j'(z_{4_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-4}^{(2)}
    \end{array}
    \right]
}_{\text{delta_hidden}}
=
\left[
\begin{array}{cccc}
      a^{(0)}_{1_{n}} g_j'(z_{1_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}
     &a^{(0)}_{1_{n}} g_j'(z_{2_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-2}^{(2)}
     &a^{(0)}_{1_{n}} g_j'(z_{3_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-3}^{(2)}
     &a^{(0)}_{1_{n}} g_j'(z_{4_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-4}^{(2)}
     \\ 
      a^{(0)}_{2_{n}} g_j'(z_{1_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}
     &a^{(0)}_{2_{n}} g_j'(z_{2_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-2}^{(2)}
     &a^{(0)}_{2_{n}} g_j'(z_{3_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-3}^{(2)}
     &a^{(0)}_{2_{n}} g_j'(z_{4_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-4}^{(2)}
\end{array}
\right]
$$

$$
\underbrace{
\overrightarrow{
\frac{\partial E}{\partial b_{j_{n}}}
}
}_{\text{bias_gradients['b_1']}}
=
\left[
\begin{array}{cccc}
     \frac{\partial E}{\partial b_{j1}}_{n}\\
     \frac{\partial E}{\partial b_{j2}}_{n}\\
     \frac{\partial E}{\partial b_{j3}}_{n}\\
     \frac{\partial E}{\partial b_{j4}}_{n}
\end{array}
\right]
= \vec{\delta_{j_{n}}}
=
\underbrace{
    \left[
    \begin{array}{cccc}
       g_j'(z_{1_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-1}^{(2)}\\
       g_j'(z_{2_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-2}^{(2)}\\
       g_j'(z_{3_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-3}^{(2)}\\
       g_j'(z_{4_{n}}^{(1)})(t_n - a_{1_{n}}^{(2)})\cdot g_k'(z_{1_{n}}^{(2)})\cdot\omega_{1-4}^{(2)}
    \end{array}
    \right]
}_{\text{delta_hidden}}
$$

In [ ]:
def step_III_gradient_calculation(
    delta_output, delta_hidden, a_hidden, network_input, weight_gradients, bias_gradients
):
    if 'w_2' in weight_gradients:  # multi-layer network
        weight_gradients['w_2'] = a_hidden.T * delta_output
        bias_gradients['b_2'] = delta_output * 1
        weight_gradients['w_1'] = network_input.T * delta_hidden
        bias_gradients['b_1'] = delta_hidden * 1
    else:  # single-layer network
        weight_gradients['w_1'] = network_input.T * delta_output
        bias_gradients['b_1'] = delta_output * 1

    return weight_gradients, bias_gradients

In [ ]:
for iteration in range(n_iterations):
    obs_error = []
    print(f"iteration: {iteration}")
    for i, (network_input, target) in enumerate(zip(observations, targets)):
        network_input = np.atleast_2d(network_input)
        
        # Step I: Forward propagate input signal through the network,
        # collecting activations and hidden states
        a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(
            network_input, weights, biases, g_activation
        )
        
        # Step II: Backpropagate error signal
        delta_output, delta_hidden = step_II_backprop(
            target, a_output, z_output, z_hidden, weights, g_activation_prime
        )
        
        # Step III. Calculate Error gradient w.r.t. parameters
        weight_gradients, bias_gradients = step_III_gradient_calculation(
            delta_output, delta_hidden, a_hidden, network_input,
            weight_gradients, bias_gradients
        )
        
        if i==iteration: # random
            print(f"i: {i}")
            display_htmlFcn2( 
                             "g_activation_prime['g_out'](z_output) * (a_output - target)", # delta_output
                             "bias_gradients['b_1']", # delta_output
                             "network_input",
                             "network_input.T * delta_output", # weight_gradients['w_1']
                             _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=2)

#### Step VI code

**Step IV: update parameters**

To update the model parameters based on the gradients calculated in Step III. Note that the gradients point in the direction in parameter space that will increase the value of the error function. Thus when updating the model parameters we should choose to go in the opposite direction. How far do we travel in that direction? That is generally determined by a user-defined step size–aka learning rate–parameter, $\eta$. Thus, given the parameter gradients and the step size, the weights and biases for a given layer are updated accordingly:

$$
\begin{align}
w_{l-1,l} &\leftarrow w_{l-1,l} - \eta \frac{\partial E}{\partial w_{l-1, l}} \\ 
b_l &\leftarrow b_{l} - \eta \frac{\partial E}{\partial b_{l}}
\end{align}
$$

In [ ]:
def step_IV_update_parameters(weights, biases, weight_gradients, bias_gradients, learning_rate):
    if 'w_2' in weights:  # multi-layer network
        weights['w_2'] = weights['w_2'] - weight_gradients['w_2'] * learning_rate
        biases['b_2'] = biases['b_2'] - bias_gradients['b_2'] * learning_rate

    weights['w_1'] = weights['w_1'] - weight_gradients['w_1'] * learning_rate
    biases['b_1'] = biases['b_1'] - bias_gradients['b_1'] * learning_rate
    return weights, biases

In [ ]:
for iteration in range(n_iterations):
    obs_error = []
    print(f"iteration: {iteration}")
    for i, (network_input, target) in enumerate(zip(observations, targets)):
        network_input = np.atleast_2d(network_input)
        
        # Step I: Forward propagate input signal through the network,
        # collecting activations and hidden states
        a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(
            network_input, weights, biases, g_activation
        )
        
        # Step II: Backpropagate error signal
        delta_output, delta_hidden = step_II_backprop(
            target, a_output, z_output, z_hidden, weights, g_activation_prime
        )
        
        # Step III. Calculate Error gradient w.r.t. parameters
        weight_gradients, bias_gradients = step_III_gradient_calculation(
            delta_output, delta_hidden, a_hidden, network_input,
            weight_gradients, bias_gradients
        )
        
        # Step IV. Update model parameters using gradients
        weights_orig = copy.copy(weights)
        biases_orig  = copy.copy(biases)
        weights, biases = step_IV_update_parameters(
            weights, biases, weight_gradients, bias_gradients, learning_rate
        )
        if i==iteration: # random
            print(f"i: {i}")
            display_htmlFcn2("weights_orig['w_1']", "weight_gradients['w_1']", "[learning_rate]",
                             "weights_orig['w_1'] - weight_gradients['w_1'] * learning_rate", "weights['w_1']",  
                             "biases_orig['b_1']", "bias_gradients['b_1']", "[learning_rate]",
                             "biases_orig['b_1'] - bias_gradients['b_1'] * learning_rate", "biases['b_1']",
                             _localVars=locals(), _applyOrApplyMap=softZeros, _float_prec=2)

#### Whole function

Rest of `run_ann_training_simulation()` concerns:
 - plot: `get_prediction_surface()`, `get_prediction_series()`, 
```python 
weights_history[iteration] = copy.copy(weights)
biases_history[iteration] = copy.copy(biases)
loss_history.append(sum(obs_error))
if problem_type in ('SIN', 'ABS'):
    prediction_history[iteration] = get_prediction_series(prediction_x, weights, biases, g_activation)
else:                              
    prediction_history[iteration] = get_prediction_surface(prediction_surface_xy, weights, biases, g_activation)
``` 
 - learning rate update: 
 
```python
for iteration in range(n_iterations):
    obs_error = []
    for network_input, target in zip(observations, targets):
        ...
    learning_rate *= .95
```

In [ ]:
def get_prediction_surface(pred_surface_xy, weights, biases, g_activation):
    """Calculates current prediction surface for classification problem. Used for visualization"""
    prediction_surface = [step_I_forwardprop(xy, weights, biases, g_activation)[0] for xy in pred_surface_xy]
    return np.array(prediction_surface).squeeze().reshape(PREDICTION_SURFACE_RESOLUTION, PREDICTION_SURFACE_RESOLUTION)

def get_prediction_series(pred_x, weights, biases, g_activation):
    """Calculates current prediction series for regression problem. Used for visualization"""
    return step_I_forwardprop(pred_x[:, None], weights, biases, g_activation)[0]

def run_ann_training_simulation(
    problem_type='AND',
    n_hidden_units=0,
    n_iterations=100,
    n_observations=50,
    learning_rate=3
):
    """Simulate ANN training on one of the following problems:

    Binary Classification:
        "AND": noisy binary logical AND data distrubted as 2D datapoints
        "OR": noisy binary logical OR data distrubted as 2D datapoints
        "XOR": noisy binary logical XOR data distrubted as 2D datapoints
        "RING": data are a mode of one binary class surronded by a ring of the other
    Regression (2D)
        "SIN": data are noisy observations around the sin function with a slight vertical offset
        "ABS": data are noisy observations around teh absolute value function

    Parameters
    ----------
    problem_type : str
        One of the problem types listed above
    n_hidden_units : int
        The number of hidden units in the hidden layer. Zero indicates no hidden layer
    n_iterations : int
        The number of times to run through the training observations
    n_observations : int
        The number of data points or (or dataset replicas for classification) that are used
        in the training dataset
    learning_rage : float
        The initial learning rate (annealing is applied at each iteration)

    Returns
    -------
    loss_history : list[float]
        The loss function at each iteration of training
    prediction_history : dict
        Network predictions over the range of the training input. Used for learning visualization.
        Keys are are the iteration number. Values are either prediction surface for classification
        problems, or prediction series for regression
    weights_history : dict
        For each iteration, a snapshot of the state of the parameters. Used for visualizing hidden
        unit states at each iteration
    biases_history : dict
        For each iteration, a snapshot of the state of the biases. Used for visualization. Used for
        visualizing hidden unit states at each iteration
    """

    # Initialize problem data
    if problem_type in ('SIN', 'ABS'):
        observations, targets = generate_regression_data(problem_type, n_observations)
    else:
        obs_x, obs_y, targets = generate_classification_data(problem_type, n_observations)
        observations = np.vstack([obs_x, obs_y]).T

    # Initialize model parameters $\theta$
    n_output_dims = 1
    n_obs, n_input_dims = observations.shape
    weights, biases, weight_gradients, bias_gradients = initialize_network_parameters(
        n_input_dims, n_hidden_units, n_output_dims
    )

    # Initialize problem-specific activation functions and their derivatives
    g_activation = {}
    g_activation_prime = {}
    if problem_type in ('SIN', 'ABS'):  # regression using linear output (and optional tanh hidden) activations
        g_activation['g_out'], g_activation_prime['g_out'], _ = activation_functions['linear']
        if 'w_2' in weights:
            g_activation['g_1'], g_activation_prime['g_1'], _ = activation_functions['tanh']
    else:  # classification using all sigmoid activations
        g_activation['g_out'], g_activation_prime['g_out'], _ = activation_functions['sigmoid']
        if 'w_2' in weights:
            g_activation['g_1'], g_activation_prime['g_1'], _ = activation_functions['sigmoid']

    # Setup for learning history / visualization
    loss_history = []
    prediction_history = {}
    weights_history = {}
    biases_history = {}
    if problem_type in ('SIN', 'ABS'):
        prediction_x = np.linspace(-5, 5, PREDICTION_SURFACE_RESOLUTION)
    else:
        prediction_surface_range = np.linspace(-.5, 1.5, PREDICTION_SURFACE_RESOLUTION)
        prediction_surface_x, prediction_surface_y = np.meshgrid(prediction_surface_range, prediction_surface_range)
        prediction_surface_xy = [(x, y) for x, y in zip(prediction_surface_x.ravel(), prediction_surface_y.ravel())]

    # Run the training
    for iteration in range(n_iterations):
        obs_error = []
        for network_input, target in zip(observations, targets):
            network_input = np.atleast_2d(network_input)

            # Step I: Forward propagate input signal through the network,
            # collecting activations and hidden states
            a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(
                network_input, weights, biases, g_activation
            )

            # Step II: Backpropagate error signal
            delta_output, delta_hidden = step_II_backprop(
                target, a_output, z_output, z_hidden, weights, g_activation_prime
            )

            # Step III. Calculate Error gradient w.r.t. parameters
            weight_gradients, bias_gradients = step_III_gradient_calculation(
                delta_output, delta_hidden, a_hidden, network_input,
                weight_gradients, bias_gradients
            )

            # Step IV. Update model parameters using gradients
            weights, biases = step_IV_update_parameters(
                weights, biases, weight_gradients, bias_gradients, learning_rate
            )

            # Keep track of observation error for loss history
            obs_error.append(error_function(a_output, target))

        # Anneal the learning rate (helps learning)
        learning_rate *= .95

        # Keep learning history for visualization
        weights_history[iteration] = copy.copy(weights)
        biases_history[iteration] = copy.copy(biases)
        loss_history.append(sum(obs_error))
        if problem_type in ('SIN', 'ABS'):
            prediction_history[iteration] = get_prediction_series(prediction_x, weights, biases, g_activation)
        else:
            prediction_history[iteration] = get_prediction_surface(prediction_surface_xy, weights, biases, 
                                                                   g_activation)
    return loss_history, prediction_history, weights_history, biases_history

Auxiliar functions and contstants:

In [ ]:
def error_function(prediction, target=1):
    """Squared error function (f(x) - y)**2"""
    return (prediction - target)**2

#### Example

Let's see the output of `run_ann_training_simulation` in an ANN without hidden layers that performs an $OR$ classification problem:

In [ ]:
N_HIDDEN_UNITS = 0
PROBLEM_TYPE = 'OR'
n_iterations=100
loss_history, prediction_history, weights_history, biases_history = run_ann_training_simulation(
    problem_type=PROBLEM_TYPE,
    n_hidden_units=N_HIDDEN_UNITS,
    n_iterations=n_iterations,
    learning_rate=2,
)

In [ ]:
loss_history_1D = np.array(loss_history).flatten()

In [ ]:
w = [weights_history.get(i).get('w_1', []) for i in range(n_iterations)]
w1 = [k[0][0] for k in w]
w2 = [k[1][0] for k in w]
b = [biases_history.get(i).get('b_1')[0][0] for i in range(n_iterations)]

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, figsize=(12, 4))

y = copy.deepcopy(loss_history_1D)
n_obs = 200 # {run_ann_training_simulation() has default n_observations=50} & {OR} --> len(obs_x) = 4*50 = 200
y /= n_obs # mean error in each input. Revert loss_history.append(sum(obs_error)) of run_ann_training_simulation()

i_split = 25
nan1 = np.full(n_iterations - i_split, np.NaN)
nan2 = np.full(i_split, np.NaN)
y1 = np.concatenate((y[:i_split], nan1))
y2 = np.concatenate((nan2, y[i_split:]))

# different scales
axs[0,0].semilogy(y1)
axs[0,0].semilogy(y2, '--', color='g')
axs[1,0].semilogy(y2, color='g')
axs[0,0].set_title('Mean Error')
axs[1,0].set_ylabel(f'Zoomed from iter {i_split} on')
axs[1,0].set_xlabel('Iter')

axs[0,1].semilogy(w1, label='$w_{i_{1}j_{1}}$')
axs[0,1].semilogy(w2, label='$w_{i_{2}j_{1}}$')
axs[1,1].plot(b, label='$b_{j_{1}}$')
axs[0,1].legend()
axs[1,1].legend()

plt.tight_layout(h_pad=0)

In [ ]:
display_htmlFcn2("loss_history_1D[:5]", "prediction_history[0][:5]", "[list(prediction_history.keys())[i] for i in [0,1,2,-2,-1]]",
                 _localVars=locals(), _float_prec=[1,1,0])

In [ ]:
i = list(weights_history.keys())[-1] # key_last_iter
w = weights_history.get(i).get('w_1', [])
b = biases_history.get(i).get('b_1', [])
w = np.array(w)[np.newaxis,:]
plotNN01([2,1], weights=w, biases=b, figsize=(4,3), 
         arrow_color_fcn=lambda x: "green" if x > 0 else "red",arrow_width_variable = True,arrow_width = 0.001*50);

In [ ]:
w = weights_history.get(i)
b = biases_history.get(i)

In [ ]:
def get_observations_and_g(problem_type=PROBLEM_TYPE, n_obs_per_class=5):
    # Initialize problem data
    obs_x, obs_y, targets = generate_classification_data(problem_type, n_obs_per_class)
    observations = np.vstack([obs_x, obs_y]).T

    # Initialize problem-specific activation functions and their derivatives
    g_activation = {}
    g_activation_prime = {}
    g_activation['g_out'], g_activation_prime['g_out'], _ = activation_functions['sigmoid']
    
    return observations, targets, g_activation, g_activation_prime

observations, targets, g_activation, g_activation_prime = get_observations_and_g()

Test weights and biases manually:

In [ ]:
_targets = [0,1,1,1]
_observations = np.array([[0,0],[0,1],[1,0],[1,1]])
for o,t in zip(_observations,_targets):
    z = o @ w['w_1'] + b['b_1']
    prediction = g_activation['g_out'](z[0])[0]
    print(f"inputs {o} should output {t}\tbut errors {(prediction - t)**2:.1e}")

Test weights and biases programmatically:

In [ ]:
def NN_errors_and_output(observations=observations, targets=targets, weights=w, biases=b, g_activation=g_activation):

    a_outputArr = []
    errorArr = []
    for i, (network_input, target) in enumerate(zip(observations, targets)):
        network_input = np.atleast_2d(network_input)

        # Step I: Forward propagate input signal through the network,
        # collecting activations and hidden states
        a_output, z_output, a_hidden, z_hidden = step_I_forwardprop(
            network_input, weights, biases, g_activation
        )

        error = error_function(a_output, target) # (prediction - target)**2

        a_outputArr.append([k[0] for k in a_output])
        errorArr.append([k[0] for k in error])

    return a_outputArr, np.array(errorArr).flatten()

def plot_NN_errors(errorArr, ax, set_x=False):
    n = len(errorArr)
    bin_edges = np.logspace(np.log10(min(errorArr)), np.log10(max(errorArr)), num=int(n/4)) # log spaced bin edges
    ax.hist(errorArr, bins=bin_edges, alpha=0.5, histtype='stepfilled', color='steelblue', edgecolor='k');
    ax.set_xscale('log')
    ax.set_ylabel('Frequency')
    ax.set_title(f'observations = {n}') 
    ax.tick_params(labelsize=7); ax.set_yticks(ax.get_yticks()[::2])
    if set_x:
        ax.set_xlabel('Error (log scale)')
    return ax

a_outputArr, errorArr = NN_errors_and_output()

observations_big, targets_big, _, _ = get_observations_and_g(n_obs_per_class=200)
a_output_big, errorArr_big = NN_errors_and_output(observations_big, targets_big)

plt.ioff() # prevent plots from being displayed in the output of Jupyter Notebook
fig, axs = plt.subplots(2,1, sharex=True) 
axs[0] = plot_NN_errors(errorArr, axs[0])
axs[1] = plot_NN_errors(errorArr_big, axs[1], set_x=True)
plt.tight_layout(h_pad=2)
plt.ion() # revert plt.ioff()
html_plot_fig = display_htmlFcn_plots(fig, width=500.0, height=500.0, caption="errors of each single observation")

print(f"PROBLEM_TYPE: {PROBLEM_TYPE}")
display_htmlFcn2("observations", "np.rint(observations)", "targets", "a_outputArr", "errorArr", html_plot_fig,
                 _localVars=locals(), _applyOrApplyMap=[None,softZeros,softZeros,None,{"apply":highlight_maxFcn}],
                 _formatVal=["{:.1f}","{:.0f}","{:.0f}","{:.1e}"])

### `visualize_classification_learning`

In [ ]:
ann_main_classif02 = lambda *args, **kwargs: ann_main_classif(generate_classification_data,
                                                            run_ann_training_simulation,
                                                            plotNN02,
                                                            *args, **kwargs)

#### `AND` example

In [ ]:
%%time
ann_main_classif02('AND', N_HIDDEN_UNITS = 0)

#### `XOR` example

What about more complex categorization criterion that cannot be represented by a single plane? An example of a more complex binary classification criterion is the XOR operator. Here we can **NOT** achieve an overall **low error** if `N_HIDDEN_UNITS = 1`.

In [ ]:
%%time
ann_main_classif02('XOR', N_HIDDEN_UNITS = 0)

In [ ]:
%%time
ann_main_classif02('XOR', N_HIDDEN_UNITS = 1)

Below we instead train a two-layer (i.e. single-hidden-layer) neural network on the XOR dataset. The network incorporates a hidden layer with 4 hidden units  (`N_HIDDEN_UNITS = 4`) and logistic sigmoid activation functions for all units in the hidden and output layers.

In [ ]:
%%time
ann_main_classif02('XOR', N_HIDDEN_UNITS = 4)

Final weights and biases are:

In [ ]:
loss_history, prediction_history, weights_history, biases_history = run_ann_training_simulation(
    problem_type='XOR',
    n_hidden_units=4,
    n_iterations=100,
    learning_rate=2,
)

In [ ]:
plotNN02(weights_history, biases_history);

In [ ]:
plotNN02(weights_history, biases_history, 
        kwargs=dict(arrow_color_fcn=lambda x: "green" if x > 0 else "red",
                    arrow_width_variable = True,arrow_width = 0.001*50,
                   showFormulas=False));

#### `RING` example

Figure 12 shows the results for learning a even more difficult nonlinear categorization function: points in and around $(x1,x2)=(0.5,0.5)$ are categorized as 1, while points in a ring surrounding the 1 datapoints are categorized as a 0:

Note: in [website](https://dustinstansbury.github.io/theclevermachine/a-gentle-introduction-to-neural-networks) it uses `learning_rate=2`, but actually it converges with `learning_rate=4`.

In [ ]:
ann_main_classif02('RING', N_HIDDEN_UNITS = 4, n_iterations=100, learning_rate=4)

In [ ]:
loss_history, prediction_history, weights_history, biases_history = run_ann_training_simulation(
    problem_type='RING',
    n_hidden_units=4,
    n_iterations=100,
    learning_rate=2,
)
plotNN02(weights_history, biases_history, 
        kwargs=dict(arrow_color_fcn=lambda x: "green" if x > 0 else "red",
                    arrow_width_variable = True,arrow_width = 0.001*50,
                   showFormulas=False));